<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuningDM_MISTRAL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q
!pip install torch -q
!pip install accelerate -q
!pip install bitsandbytes -q
!pip install datetime -q
!pip install unsloth -q
!pip install openai -q
!pip install openai-agents -q

## mistral with transformer

In [2]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

! pip install peft --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## code2dataset

https://medium.com/ai-simplified-in-plain-english/ai-driven-disruption-management-for-next-gen-flight-scheduling-using-openai-51c7064af3e5

In [2]:
import json
dataset = [
    {
        "agent_name": "CrewAgent",
        "code": """
    class CrewAgent:
        def __init__(self, db):
            self.db = db  # Database connection

        def assign_crew_to_flight(self, crew_id, flight_id):
            \"\"\"Assigns a crew member to a flight.\"\"\"
            # Code to update database with crew assignment
            print(f"Crew {crew_id} assigned to flight {flight_id}")

        def update_crew_schedule(self, crew_id, new_schedule):
            \"\"\"Updates the schedule for a crew member.\"\"\"
            # Code to update crew schedule in the database
            print(f"Crew {crew_id} schedule updated to {new_schedule}")

        def reassign_crew(self, flight_id, disruption_type):
            \"\"\"Reassigns crew members due to a disruption.\"\"\"
            # Code to handle crew reassignment logic
            print(f"Crew reassigned for flight {flight_id} due to {disruption_type}")

        # ... other CrewAgent methods ...
    """,
        "description": """
    The CrewAgent class is responsible for managing crew-related operations in the flight disruption management system.
    It handles tasks such as assigning crew members to flights, updating crew schedules, and coordinating crew reassignments during disruptions.
    The CrewAgent interacts with the database to store and retrieve crew information.
    """
    },
    {
        "agent_name": "PassengerAgent",
        "code": """
    class PassengerAgent:
        def __init__(self, db):
            self.db = db  # Database connection

        def get_passenger_manifest(self, flight_id):
            \"\"\"Retrieves the passenger manifest for a flight.\"\"\"
            # Code to query the database for passenger list
            print(f"Passenger manifest for flight {flight_id} retrieved")
            return ["PassengerA", "PassengerB", ...]

        def notify_passenger(self, passenger_id, message):
            \"\"\"Sends a notification to a passenger.\"\"\"
            # Code to send notification (e.g., email, SMS)
            print(f"Passenger {passenger_id} notified: {message}")

        def rebook_passenger(self, passenger_id, new_flight_id):
            \"\"\"Rebooks a passenger on a new flight.\"\"\"
            # Code to update the booking information in the database
            print(f"Passenger {passenger_id} rebooked on flight {new_flight_id}")

        # ... other PassengerAgent methods ...
    """,
        "description": """
    The PassengerAgent class is responsible for handling passenger-related operations in the flight disruption management system.
    It manages tasks such as retrieving passenger manifests, notifying passengers of changes or disruptions, rebooking passengers on new flights,
    and communicating with passengers.
    """
    },
    {
        "agent_name": "FleetAgent",
        "code": """
    class FleetAgent:
        def __init__(self, db, crew_agent, passenger_agent):
            self.db = db  # Database connection
            self.crew_agent = crew_agent
            self.passenger_agent = passenger_agent

        def assess_disruption_impact(self, disruption_event):
            \"\"\"Analyzes the impact of a disruption event.\"\"\"
            # Code to analyze delayed flights, affected passengers, etc.
            print(f"Disruption impact assessed: {disruption_event}")
            return {"delayed_flights": 10, "affected_passengers": 200}

        def orchestrate_recovery(self, disruption_event):
            \"\"\"Orchestrates the recovery efforts after a disruption.\"\"\"
            # Code to coordinate crew reassignment, passenger rebooking, etc.
            print(f"Recovery orchestrated for disruption: {disruption_event}")

        def reschedule_flights(self, disruption_event):
            \"\"\"Reschedules flights to minimize disruption.\"\"\"
            # Code to generate new flight schedules
            print(f"Flights rescheduled after disruption: {disruption_event}")

        # ... other FleetAgent methods ...
    """,
        "description": """
    The FleetAgent class is responsible for orchestrating the overall recovery strategy in case of flight disruptions.
    It assesses the impact of disruptions, coordinates crew and passenger actions through the CrewAgent and PassengerAgent,
    and reschedules flights to minimize the impact of disruptions.
    """
    },
    {
        "agent_name": "Database Operations",
        "code": """
def create_database(db_name):
    \"\"\"Creates a new SQLite database.\"\"\"
    # Code to create a database file
    print(f"Database {db_name} created")

def connect_to_database(db_name):
    \"\"\"Establishes a connection to the SQLite database.\"\"\"
    # Code to connect to the database
    print(f"Connected to database {db_name}")
    return sqlite3.connect(db_name)

def close_database_connection(db_connection):
    \"\"\"Closes the connection to the SQLite database.\"\"\"
    # Code to close the database connection
    print("Database connection closed")
    db_connection.close()
""",
        "description": """
    These functions handle database operations for the flight disruption management system.
    They include creating a new SQLite database, connecting to the database, and closing the database connection.
    """
    },
    {
        "agent_name": "Analytics Functions",
        "code": """
def analyze_disruption_impact(disruption_event):
    \"\"\"Analyzes the impact of a disruption event.\"\"\"
    # Code to analyze delayed flights, affected passengers, etc.
    print(f"Disruption impact analysis for: {disruption_event}")
    return {"delayed_flights": 15, "affected_passengers": 300}

def calculate_disruption_cost(delayed_flights):
    \"\"\"Calculates the estimated cost of a disruption.\"\"\"
    # Code to calculate disruption cost based on delays
    cost_per_flight = 1000  # Example cost per delayed flight
    total_cost = delayed_flights * cost_per_flight
    print(f"Disruption cost: ${total_cost}")
    return total_cost
""",
        "description": """
    These functions provide analytics capabilities for the flight disruption management system.
    They include analyzing the impact of disruption events and calculating the estimated cost of disruptions.
    """
    },
    {
        "agent_name": "Disruption Simulation",
        "code": """
def simulate_disruption(db, airport_code, start_time, end_time):
    \"\"\"Simulates a disruption event at a specific airport.\"\"\"
    # Code to introduce delays or cancellations in the database
    print(f"Disruption simulated at {airport_code} from {start_time} to {end_time}")

    # Example: Introduce a delay to a flight
    # Assuming you have a function to fetch flights and update their status
    # flight_to_delay = get_flight_at_airport(db, airport_code, start_time)
    # if flight_to_delay:
    #     update_flight_status(db, flight_to_delay['flight_id'], 'delayed')
    return "Simulated disruption event"
""",
        "description": """
    This function simulates a disruption event at a specific airport within a defined time frame.
    It introduces delays or cancellations to flights in the database to mimic real-world disruptions.
    """
    }
]

# Save the dataset to a JSON file
with open("flight_agent_code_dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

print("Dataset created and saved to flight_agent_code_dataset.json")

Dataset created and saved to flight_agent_code_dataset.json


In [3]:
import json

with open("flight_agent_code_dataset.json", "r") as f:
    dataset = json.load(f)

print(f"Dataset loaded with {len(dataset)} entries")
# Now 'dataset' is a list of dictionaries, just like before

Dataset loaded with 6 entries


## fine tuning

In [15]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from torch.nn.utils.rnn import pad_sequence

training_args = TrainingArguments(
    output_dir="./mistral-fine-tuned",
    #use_cache=False,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=2,
    warmup_steps=200,
    optim="adamw_torch_fused",
    num_train_epochs=5,
    max_steps=3000,
    learning_rate=1e-5,
    logging_steps=200,
    bf16=True,
    tf32=True,
    lr_scheduler_type="cosine",
    weight_decay=0.1,
    eval_steps=10,  # Assuming you want eval_steps to be 10
    report_to="none",
    save_steps=20,
    max_grad_norm=1.0,
    #metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_strategy="steps",
    #evaluation_strategy="steps",

    #greater_is_better=False,  # For minimizing eval_loss (accuracy is maximized by default)

    #no_cuda=False if torch.cuda.is_available() else True # Add this line if needed for CPU training

)

In [16]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
#from unsloth import FastLanguageModel
from torch.nn.utils.rnn import pad_sequence


# 1. Load the Dataset
with open("flight_agent_code_dataset.json", "r") as f:
    dataset = json.load(f)

print(f"Dataset loaded with {len(dataset)} entries")

# 2. Load Tokenizer and Model (Unsloth)


# 3. Tokenize the Dataset and Create Labels
# Tokenize the dataset and create labels
def tokenize_function(examples):
    tokenized_output = tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)
    input_ids = tokenized_output["input_ids"]
    attention_mask = tokenized_output["attention_mask"]

    # Create labels by shifting input_ids
    labels = input_ids[1:].copy()
    labels.append(tokenizer.eos_token_id)
    item["input_ids"] = input_ids[:-1]
    item["attention_mask"] = attention_mask[:-1]
    item["labels"] = labels

    return tokenized_output  # Return the tokenized output

for item in dataset:
    tokenized_output = tokenize_function({"code": item["code"]})
    item["input_ids"] = tokenized_output["input_ids"]
    item["attention_mask"] = tokenized_output["attention_mask"]
    item["labels"] = item["input_ids"].copy()  # Or your label creation logic


print("Dataset tokenized and labels created using tokenizer")


# Data collator (ensure it handles padding if necessary or use the default)
def data_collator(data):
    input_ids = [item['input_ids'] for item in data]
    attention_mask = [item['attention_mask'] for item in data]
    labels = [item['labels'] for item in data]

    # Pad the sequences manually if needed, or rely on the tokenizer's padding
    input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(ids) for ids in input_ids], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(label) for label in labels], batch_first=True, padding_value=-100)  # Or your padding value

    return {'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels}


# 4. Configure PEFT (LoRA)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 5. Define Training Arguments
training_args_old = TrainingArguments(
    output_dir="./mistral-fine-tuned",
    num_train_epochs=20,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,

    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    report_to="none"
)
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator, # Use the defined data_collator
    #callbacks=[early_stopping_callback],
)

trainer.train()

print("Mistral fine-tuning complete!")

# 7. Save the Fine-tuned Model
model.save_pretrained("./mistral-fine-tuned")
tokenizer.save_pretrained("./mistral-fine-tuned")

print("Fine-tuned model and tokenizer saved!")

Step,Training Loss
200,1.959700
400,0.771000
600,0.757400
800,0.756800
1000,0.756700
1200,0.756000
1400,0.755700
1600,0.755700
1800,0.755700
2000,0.755600


Mistral fine-tuning complete!
Fine-tuned model and tokenizer saved!


## evaluation

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import warnings
warnings.filterwarnings("ignore")

# Load the fine-tuned model and tokenizer
model_path = "./mistral-fine-tuned"  # Path to your saved model
base_model_path = "mistralai/Mistral-7B-Instruct-v0.1" # The original base model

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config  # If you used quantization, include this
)

# Load the PEFT adapter and apply it to the base model
model = PeftModel.from_pretrained(base_model, model_path)

tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# Evaluation Function (Code Generation)
def evaluate_code_generation(model, tokenizer, prompt):
    """
    Evaluates the model's code generation for a given prompt.

    Args:
        model: The fine-tuned model.
        tokenizer: The tokenizer.
        prompt: A string describing the desired code functionality.

    Returns:
        generated_code: The code generated by the model.
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=200)  # Adjust max_length as needed
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# Evaluation Function (Simulation - Simplified Example)
def evaluate_simulation(generated_code, scenario):
    """
    Evaluates the generated code in a simplified simulation.

    Args:
        generated_code: The code generated by the model.
        scenario: A dictionary representing a disruption scenario.

    Returns:
        results: A dictionary containing evaluation metrics.
    """
    # **Note:** This is a simplified example. A real simulation would involve
    # executing the generated code in a controlled environment and
    # measuring its effects.

    # In this example, we'll just check if the generated code contains
    # keywords related to handling the scenario.

    results = {}
    if "delay" in scenario and "reassign_crew" in generated_code:
        results["crew_reassignment"] = "Handled"
    else:
        results["crew_reassignment"] = "Not Handled"

    if "passenger" in scenario and "rebook_passenger" in generated_code:
        results["passenger_rebooking"] = "Handled"
    else:
        results["passenger_rebooking"] = "Not Handled"

    return results

# --- Example Usage ---

# 1. Code Generation Evaluation
prompt = "Generate Python code for a CrewAgent to reassign crew members due to a weather delay."
generated_code = evaluate_code_generation(model, tokenizer, prompt)
print("Generated Code:\n", generated_code)

# 2. Simulation Evaluation (Simplified)
scenario = {"delay": True, "passenger": True}  # Example scenario: weather delay affecting passengers
simulation_results = evaluate_simulation(generated_code, scenario)
print("Simulation Results:\n", simulation_results)

## evaluation2

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import warnings

warnings.filterwarnings("ignore")

# Load the fine-tuned model and tokenizer
model_path = "./mistral-fine-tuned"
base_model_path = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the base model
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        # quantization_config=bnb_config
    )
except Exception as e:
    print(f"Error loading base model: {e}")
    exit()

# Load the PEFT adapter
try:
    model = PeftModel.from_pretrained(base_model, model_path)
except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    exit()

tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# Evaluation Function (Code Generation)
def evaluate_code_generation(model, tokenizer, prompt, max_length=200):
    """
    Evaluates the model's code generation for a given prompt.

    Args:
        model: The fine-tuned model.
        tokenizer: The tokenizer.
        prompt: A string describing the desired code functionality.
        max_length: Maximum length of the generated code.

    Returns:
        generated_code: The code generated by the model.
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    try:
        outputs = model.generate(**inputs, max_length=max_length)
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_code
    except Exception as e:
        print(f"Error during code generation: {e}")
        return ""

# Evaluation Function (Simulation - Simplified Example)
def evaluate_simulation(generated_code, scenario):
    """
    Evaluates the generated code in a simplified simulation.

    Args:
        generated_code: The code generated by the model.
        scenario: A dictionary representing a disruption scenario.

    Returns:
        results: A dictionary containing evaluation metrics.
    """
    # **Note:** This is a simplified example. A real simulation would involve
    # executing the generated code in a controlled environment and
    # measuring its effects.

    # Check for keywords related to handling the scenario.
    results = {}
    results["crew_reassignment"] = "Handled" if "delay" in scenario and "reassign_crew" in generated_code else "Not Handled"
    results["passenger_rebooking"] = "Handled" if "passenger" in scenario and "rebook_passenger" in generated_code else "Not Handled"
    results["flight_rescheduling"] = "Handled" if "delay" in scenario and "reschedule_flights" in generated_code else "Not Handled"
    results["disruption_assessment"] = "Handled" if "delay" in scenario and "assess_disruption_impact" in generated_code else "Not Handled"
    results["recovery_orchestration"] = "Handled" if "delay" in scenario and "orchestrate_recovery" in generated_code else "Not Handled"
    return results

# --- Example Usage ---
# 1. Code Generation Evaluation
prompt = "Generate Python code to handle a flight disruption caused by a weather delay, including crew reassignment, passenger rebooking, and flight rescheduling."
generated_code = evaluate_code_generation(model, tokenizer, prompt)
print("Generated Code:\n", generated_code)

# 2. Simulation Evaluation (Simplified)
scenario = {"delay": True, "passenger": True}
simulation_results = evaluate_simulation(generated_code, scenario)
print("Simulation Results:\n", simulation_results)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Code:
 Generate Python code to handle a flight disruption caused by a weather delay, including crew reassignment, passenger rebooking, and flight rescheduling.

```python
# Flight class
class Flight:
    def __init__(self, flight_number, departure_airport, arrival_airport, departure_time, arrival_time):
        self.flight_number = flight_number
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.crew = []
        self.passengers = []

    # Add crew to flight
    def add_crew(self, crew_member):
        self.crew.append(crew_member)

    # Add
Simulation Results:
 {'crew_reassignment': 'Not Handled', 'passenger_rebooking': 'Not Handled', 'flight_rescheduling': 'Not Handled', 'disruption_assessment': 'Not Handled', 'recovery_orchestration': 'Not Handled'}


## attention test

In [ ]:
import torch
import xformers
print("PyTorch version:", torch.__version__)
print("xformers version:", xformers.__version__)

PyTorch version: 2.6.0+cu124
xformers version: 0.0.29.post3


In [ ]:
import torch
import xformers.ops as xops

# Example input shapes (from your error message)
batch_size = 2
seq_len = 291
num_heads = 8
num_groups = 4
d_head = 128

# Create dummy tensors
query = torch.randn(batch_size, seq_len, num_heads, num_groups, d_head, dtype=torch.float16).cuda()
key = torch.randn(batch_size, seq_len, num_heads, num_groups, d_head, dtype=torch.float16).cuda()
value = torch.randn(batch_size, seq_len, num_heads, num_groups, d_head, dtype=torch.float16).cuda()

# Create attention mask
attn_bias = xops.fmha.attn_bias.LowerTriangularMask()

try:
    # Try memory-efficient attention
    output = xops.memory_efficient_attention(query, key, value, attn_bias=attn_bias)
    print("Attention successful!")
    print("Output shape:", output.shape)
except Exception as e:
    print("Error:", e)

Attention successful!
Output shape: torch.Size([2, 291, 8, 4, 128])
